# Test

In [3]:
from pathlib import Path
from typing import Sequence, Optional, Union
import shutil

import duckdb
import pandas as pd
import pyarrow
import pyarrow.parquet
from shapely.geometry import Polygon
from shapely import contains_xy


def read_ais_df(
    csv_path: Union[Path, str],
    bbox: Sequence[float],
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Read AIS CSV into a DataFrame using DuckDB and apply basic cleaning.

    Operations:
    - Spatial bbox filter in the SQL
    - Rename & parse Timestamp
    - Drop rows with invalid timestamps
    - Check required columns exist
    - Ensure MMSI is a string

    Parameters
    ----------
    csv_path : Path | str
        Path to the AIS CSV file.
    bbox : Sequence[float]
        Bounding box as [lat_max, lon_min, lat_min, lon_max].
    verbose : bool, optional
        If True, print basic info about the loaded data.

    Returns
    -------
    pd.DataFrame
        AIS data within the given bounding box, with basic cleaning applied.

    Examples
    --------
    >>> bbox = [57.58, 10.5, 57.12, 11.92]
    >>> df_raw = read_ais_df("ais-data/aisdk-2025-11-05.csv", bbox, verbose=True)
    """
    lat_max, lon_min, lat_min, lon_max = bbox

    query = f"""
    SELECT *
    FROM read_csv_auto('{csv_path}', AUTO_DETECT=TRUE)
    WHERE Latitude <= {lat_max}
      AND Latitude >= {lat_min}
      AND Longitude >= {lon_min}
      AND Longitude <= {lon_max}
    ;
    """

    df = duckdb.query(query).to_df()

    # Rename Timestamp column and parse to datetime
    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(
        df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce"
    )

    # Drop rows where timestamp parsing failed
    df = df.dropna(subset=["Timestamp"])

    # Basic column checks
    required_columns = ["Latitude", "Longitude", "Timestamp", "MMSI", "SOG"]
    missing = [col for col in required_columns if col not in df.columns]
    if missing:
        raise KeyError(f" Required columns missing: {missing}")

    # Ensure MMSI is string for later processing
    df["MMSI"] = df["MMSI"].astype(str)

    if verbose:
        print(
            f" Read AIS data: {len(df):,} rows within bbox, "
            f" {df['MMSI'].nunique():,} unique vessels"
        )

    return df


def filter_ais_df(
    df: pd.DataFrame,
    polygon_coords: Sequence[tuple[float, float]],
    allowed_mobile_types: Optional[Sequence[str]] = ("Class A", "Class B"),
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Apply AIS filtering steps to a DataFrame.

    Steps:
    1) Filter by "Type of mobile" (default: keep only "Class A" and "Class B")
    2) MMSI sanity checks (length == 9 and MID in [200, 775])
    3) Drop duplicates on (Timestamp, MMSI)
    4) Polygon filtering using Shapely (lon, lat)

    Parameters
    ----------
    df : pd.DataFrame
        Input AIS DataFrame with at least the columns:
        ["Latitude", "Longitude", "Timestamp", "MMSI"].
    polygon_coords : Sequence[tuple[float, float]]
        Polygon vertices as (lon, lat) pairs.
    allowed_mobile_types : Sequence[str] or None, optional
        Types of mobile to keep (e.g., ["Class A", "Class B"]).
        If None, the "Type of mobile" filter is skipped (if the column exists).
    verbose : bool, optional
        If True, print detailed filtering information.

    Returns
    -------
    pd.DataFrame
        Filtered AIS DataFrame.

    Examples
    --------
    >>> polygon_coords = [
    ...     (10.5162, 57.3500),
    ...     (10.9314, 57.5120),
    ...     (11.5128, 57.5785),
    ...     (11.9132, 57.5230),
    ...     (11.9189, 57.4078),
    ...     (11.2133, 57.1389),
    ...     (11.0067, 57.1352),
    ...     (10.5400, 57.1880),
    ...     (10.5162, 57.3500),
    ... ]
    >>> df_filt = filter_ais_df(df_raw, polygon_coords, verbose=True)
    """
    df = df.copy()

    if verbose:
        print(
            f" [filter_ais_df] Before filtering: {len(df):,} rows, "
            f" [filter_ais_df] {df['MMSI'].nunique():,} unique vessels"
        )

    # ------------------------------------------------------------------
    # 1) Filter by Type of mobile (keep only selected types)
    # ------------------------------------------------------------------
    if "Type of mobile" in df.columns:
        if allowed_mobile_types is not None:
            before_rows = len(df)
            df = df[df["Type of mobile"].isin(allowed_mobile_types)].copy()

            if verbose:
                print(
                    f" [filter_ais_df] Type of mobile filtering complete: {len(df):,} rows "
                    f" [filter_ais_df] (removed {before_rows - len(df):,} rows) "
                    f" [filter_ais_df] using types: {list(allowed_mobile_types)}"
                )
        else:
            if verbose:
                print(
                    " [filter_ais_df] allowed_mobile_types is None, skipping "
                    " [filter_ais_df] 'Type of mobile' filtering step."
                )
    else:
        if verbose:
            print(" [filter_ais_df] Warning: 'Type of mobile' column not found, skipping that filter.")

    # ------------------------------------------------------------------
    # 2) MMSI sanity filters (format + MID)
    # ------------------------------------------------------------------
    # Always start from a clean string
    mmsi_str = df["MMSI"].astype(str).str.strip()

    # Valid MMSI must have 9 digits
    mask_len = mmsi_str.str.len() == 9

    # First 3 digits = MID, must be numeric and in [200, 775]
    mid = mmsi_str.str[:3]
    mask_mid = mid.str.isnumeric() & mid.astype(int).between(200, 775)

    # Combine masks
    valid_mmsi_mask = mask_len & mask_mid

    # Apply once, with aligned index
    df = df[valid_mmsi_mask].copy()

    # Update MMSI column with cleaned values
    df["MMSI"] = mmsi_str[valid_mmsi_mask]

    if verbose:
        print(
            f" [filter_ais_df] MMSI filtering complete: {len(df):,} rows, "
            f" [filter_ais_df] {df['MMSI'].nunique():,} unique vessels"
        )

    # ------------------------------------------------------------------
    # 3) Drop duplicates on (Timestamp, MMSI)
    # ------------------------------------------------------------------
    df = df.drop_duplicates(["Timestamp", "MMSI"], keep="first")

    if verbose:
        print(
            f" [filter_ais_df] Duplicate removal complete: {len(df):,} rows, "
            f" [filter_ais_df] {df['MMSI'].nunique():,} unique vessels"
        )

    # ------------------------------------------------------------------
    # 4) Polygon filtering (FAST, vectorized)
    #    NOTE: Shapely expects (x, y) = (lon, lat)
    # ------------------------------------------------------------------
    polygon = Polygon(polygon_coords)

    lons = df["Longitude"].to_numpy()
    lats = df["Latitude"].to_numpy()

    # Vectorized containment test with Shapely 2.x
    mask_poly = contains_xy(polygon, lons, lats)

    df = df[mask_poly].copy()

    if verbose:
        print(
            f" [filter_ais_df] Polygon filtering complete: {len(df):,} rows, "
            f" [filter_ais_df] {df['MMSI'].nunique():,} unique vessels"
        )

    return df


def segment_ais_tracks(
    df: pd.DataFrame,
    min_track_len: int = 256,
    min_track_duration_sec: int = 60 * 60,
    max_time_gap_sec: int = 15 * 60,
    sog_min: Optional[float] = 0.5,
    sog_max: Optional[float] = 25.0,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Filter AIS tracks and segment them by time gaps.

    Assumptions
    -----------
    - `Timestamp` is already a datetime64 dtype.
    - `SOG` is in **m/s**, and `sog_min` / `sog_max` (if given) are in m/s.

    Steps
    -----
    1) Per-MMSI track filtering:
       - length > min_track_len
       - duration >= min_track_duration_sec
       - optional SOG range [sog_min, sog_max]
    2) Sort by (MMSI, Timestamp)
    3) Define `Segment` via time gaps > `max_time_gap_sec`
    4) Apply the same filter at (MMSI, Segment) level
    5) Add `Date` column: YYYY-MM-DD

    Parameters
    ----------
    df : pd.DataFrame
        Filtered AIS DataFrame containing at least:
        ["MMSI", "Timestamp", "SOG"].
    min_track_len : int, optional
        Minimum number of points required for track/segment.
    min_track_duration_sec : int, optional
        Minimum duration in seconds for track/segment.
    max_time_gap_sec : int, optional
        Maximum allowed time gap in seconds within a segment.
    sog_min : float or None, optional
        Minimum SOG (m/s) for valid track/segment. If None, no lower bound.
    sog_max : float or None, optional
        Maximum SOG (m/s) for valid track/segment. If None, no upper bound.
    verbose : bool, optional
        If True, print information about filtering and segmentation.

    Returns
    -------
    pd.DataFrame
        DataFrame with valid tracks, including:
        - "MMSI"
        - "Timestamp"
        - "SOG"
        - "Segment" (int)
        - "Date" (str, YYYY-MM-DD)

    Examples
    --------
    >>> df_seg = segment_ais_tracks(df_filt, verbose=True)
    >>> df_seg[['MMSI', 'Timestamp', 'Segment']].head()
    """
    df = df.copy()

    required_cols = ["MMSI", "Timestamp", "SOG"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f" segment_ais_tracks: required columns missing: {missing}")

    df["MMSI"] = df["MMSI"].astype(str)
    if not pd.api.types.is_datetime64_any_dtype(df["Timestamp"]):
        raise TypeError(" segment_ais_tracks: 'Timestamp' must be a datetime dtype")

    if verbose:
        print(
            f" [segment_ais_tracks] Starting with {len(df):,} rows, "
            f" { df['MMSI'].nunique():,} unique vessels"
        )

    # helper: track filter 
    def track_filter(g: pd.DataFrame) -> bool:
        len_ok = len(g) > min_track_len

        if sog_min is not None or sog_max is not None:
            sog_max_val = g["SOG"].max()
            sog_ok = True
            if sog_min is not None:
                sog_ok &= sog_max_val >= sog_min
            if sog_max is not None:
                sog_ok &= sog_max_val <= sog_max
        else:
            sog_ok = True

        dt = (g["Timestamp"].max() - g["Timestamp"].min()).total_seconds()
        time_ok = dt >= min_track_duration_sec

        return len_ok and sog_ok and time_ok

    # ---------- 1) Filter per MMSI ----------
    df = df.groupby("MMSI", group_keys=False).filter(track_filter)

    if verbose:
        print(
            f" [segment_ais_tracks] After MMSI-level filter: {len(df):,} rows, "
            f" {df['MMSI'].nunique():,} vessels"
        )

    # ---------- 2) Sort ----------
    df = df.sort_values(["MMSI", "Timestamp"])

    # ---------- 3) Compute Segment IDs ----------
    def compute_segments(ts: pd.Series) -> pd.Series:
        gaps = ts.diff().dt.total_seconds().fillna(0)
        return (gaps >= max_time_gap_sec).cumsum()

    df["Segment"] = df.groupby("MMSI")["Timestamp"].transform(compute_segments)

    # ---------- 4) Filter per (MMSI, Segment) ----------
    df = df.groupby(["MMSI", "Segment"], group_keys=False).filter(track_filter)
    df = df.reset_index(drop=True)

    if verbose:
        print(
            f" [segment_ais_tracks] After segment-level filter: {len(df):,} rows, "
            f" {df[['MMSI','Segment']].drop_duplicates().shape[0]:,} segments"
        )

    # ---------- 5) Add Date ----------
    df["Date"] = df["Timestamp"].dt.strftime("%Y-%m-%d")

    return df


def save_by_mmsi(
    df: pd.DataFrame,
    verbose: bool = False,
    output_folder: Union[Path, str] = "ais_data_parquet",
) -> Path:
    """
    Write AIS data to a partitioned Parquet dataset.

    The output directory is **always** "ais_data_parquet".
    If it does not exist, it will be created.

    IMPORTANT
    ---------
    This function is *overwrite-safe* for the partitions present in `df`.
    For each unique (MMSI, Date, Segment) combination in `df`, the existing
    partition directory is removed before writing new data. This avoids
    accumulating multiple parquet files for the same segment when rerunning
    the pipeline for the same date/file.

    Expected columns
    ----------------
    df must contain:
    - "MMSI"    (string-like)
    - "Date"    (string, e.g. "2025-11-05")
    - "Segment" (int)

    Partition layout
    ----------------
    ais_data_parquet/
        MMSI=123456789/
            Date=2025-11-05/
                Segment=0/part-*.parquet
                Segment=1/part-*.parquet
        MMSI=987654321/
            Date=2025-11-06/
                Segment=0/part-*.parquet

    Parameters
    ----------
    df : pd.DataFrame
        Segmented AIS DataFrame containing "MMSI", "Date", "Segment".
    verbose : bool, optional
        If True, print the output path and some info.

    Returns
    -------
    Path
        Path to the root parquet dataset folder ("ais_data_parquet").

    Examples
    --------
    >>> df_seg = segment_ais_tracks(df_filt)
    >>> out_root = save_by_mmsi(df_seg, verbose=True)
    """
    df = df.copy()

    required_cols = ["MMSI", "Date", "Segment"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f" save_by_mmsi: required columns missing: {missing}")

    df["MMSI"] = df["MMSI"].astype(str)
    
    out_path = Path(output_folder)
    out_path.mkdir(parents=True, exist_ok=True)

    # ------------------------------------------------------------------
    # Remove existing segment folders for (MMSI, Date, Segment) in df
    # ------------------------------------------------------------------
    partitions = df[["MMSI", "Date", "Segment"]].drop_duplicates()

    for _, row in partitions.iterrows():
        mmsi_val = row["MMSI"]
        date_val = row["Date"]
        seg_val = row["Segment"]

        seg_dir = (
            out_path
            / f"MMSI={mmsi_val}"
            / f"Date={date_val}"
            / f"Segment={seg_val}"
        )

        if seg_dir.exists():
            if verbose:
                print(f" [save_by_mmsi] Removing existing partition: {seg_dir}")
            shutil.rmtree(seg_dir)

    # ------------------------------------------------------------------
    # Write new dataset (append is fine now that partitions are cleaned)
    # ------------------------------------------------------------------
    table = pyarrow.Table.from_pandas(df, preserve_index=False)
    pyarrow.parquet.write_to_dataset(
        table,
        root_path=str(out_path),
        partition_cols=["MMSI", "Date", "Segment"],
    )

    if verbose:
        print(f" [save_by_mmsi] Parquet dataset written/appended at: {out_path.resolve()}")

    return out_path


In [2]:
# Call functions

FOLDER_NAME = "ais-data"
OUTPUT_FOLDER_NAME = "ais-data-parquet"
DELETE_DOWNLOADED_CSV = False
verbose_mode = True

folder_path = Path(FOLDER_NAME)

csv_path = folder_path / "aisdk-2025-11-05.csv"

# [lat_max, lon_min, lat_min, lon_max]
bbox = [57.58, 10.5, 57.12, 11.92]

polygon_coords = [
    (10.5162, 57.3500),  # coast top left (lon, lat)
    (10.9314, 57.5120),  # sea top left
    (11.5128, 57.5785),  # sea top right
    (11.9132, 57.5230),  # top right (Swedish coast)
    (11.9189, 57.4078),  # bottom right (Swedish coast)
    (11.2133, 57.1389),  # sea bottom right
    (11.0067, 57.1352),  # sea bottom left
    (10.5400, 57.1880),  # coast bottom left
    (10.5162, 57.3500),  # close polygon
]

# Read raw AIS data only inside bounding box
df_raw = read_ais_df(csv_path, bbox, verbose=verbose_mode)

# Filter AIS data, keeping Class A and Class B by default,
df_filt = filter_ais_df(
    df_raw,
    polygon_coords,
    allowed_mobile_types=("Class A", "Class B"),
    verbose=verbose_mode,
)

df_seg = segment_ais_tracks(df_filt, min_track_len=256, verbose=verbose_mode)
save_by_mmsi(df_seg, verbose=verbose_mode, output_folder=OUTPUT_FOLDER_NAME)

 Read AIS data: 1,001,835 rows within bbox,  225 unique vessels
 [filter_ais_df] Before filtering: 1,001,835 rows,  [filter_ais_df] 225 unique vessels
 [filter_ais_df] Type of mobile filtering complete: 964,192 rows  [filter_ais_df] (removed 37,643 rows)  [filter_ais_df] using types: ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering complete: 964,192 rows,  [filter_ais_df] 223 unique vessels
 [filter_ais_df] Duplicate removal complete: 535,347 rows,  [filter_ais_df] 223 unique vessels
 [filter_ais_df] Polygon filtering complete: 294,200 rows,  [filter_ais_df] 157 unique vessels
 [segment_ais_tracks] Starting with 294,200 rows,  157 unique vessels
 [segment_ais_tracks] After MMSI-level filter: 186,236 rows,  124 vessels
 [segment_ais_tracks] After segment-level filter: 184,183 rows,  147 segments
 [save_by_mmsi] Removing existing partition: ais-data-parquet\MMSI=209507000\Date=2025-11-05\Segment=0
 [save_by_mmsi] Removing existing partition: ais-data-parquet\MMSI=209535000\Date=202

WindowsPath('ais-data-parquet')

# Visualization

In [5]:
from pathlib import Path
from typing import Optional, Sequence, Union, List

import duckdb
import pandas as pd
import folium
import pyarrow  # just to keep consistent with rest of file


def query_ais_duckdb(
    root_path: Union[str, Path] = "ais-data-parquet",
    dates: Optional[Union[str, Sequence[str]]] = None,
    mmsi: Optional[Union[str, Sequence[str]]] = None,
    segments: Optional[Union[int, Sequence[int]]] = None,
    columns: Optional[Sequence[str]] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Fast query helper using DuckDB to read AIS data from the partitioned
    parquet dataset generated by `ais_to_parquet`.

    Parameters
    ----------
    root_path : str or Path, optional
        Root directory of the parquet dataset (default: "ais_data_parquet").
    dates : str or list[str], optional
        Date(s) to filter on (e.g. "2025-11-05", or ["2025-11-05", "2025-11-06"]).
        If None, no date filter is applied.
    mmsi : str or list[str], optional
        MMSI or list of MMSIs to filter on. If None, no MMSI filter is applied.
    segments : int or list[int], optional
        Segment ID(s) to filter on. If None, no segment filter is applied.
    columns : list[str], optional
        Subset of columns to select. If None, selects all columns ("*").
    verbose : bool, optional
        If True, prints the generated SQL query.

    Returns
    -------
    pd.DataFrame
        Result of the DuckDB query as a pandas DataFrame.

    Examples
    --------
    >>> df = query_ais_duckdb(dates="2025-11-05")
    >>> df = query_ais_duckdb(dates="2025-11-05", mmsi="219000123")
    >>> df = query_ais_duckdb(
    ...     dates=["2025-11-05", "2025-11-06"],
    ...     mmsi=["219000123", "219000456"],
    ...     columns=["MMSI", "Timestamp", "Latitude", "Longitude"]
    ... )
    """
    root_path = Path(root_path)
    if not root_path.exists():
        raise FileNotFoundError(f"No parquet dataset found at: {root_path}")

    # Normalization helpers
    def _to_list(x) -> Optional[List]:
        if x is None:
            return None
        if isinstance(x, (str, int)):
            return [x]
        return list(x)

    def _sql_list_str(values: Sequence[str]) -> str:
        return ", ".join(f"'{v}'" for v in values)

    def _sql_list_int(values: Sequence[int]) -> str:
        return ", ".join(str(v) for v in values)

    dates_list = _to_list(dates)
    mmsi_list = _to_list(mmsi)
    segments_list = _to_list(segments)

    # Columns
    if columns is None:
        col_expr = "*"
    else:
        col_expr = ", ".join(columns)

    parquet_glob = str(root_path / "**" / "*.parquet")

    sql = f"SELECT {col_expr} FROM read_parquet('{parquet_glob}') WHERE 1=1"

    if dates_list is not None:
        sql += f" AND Date IN ({_sql_list_str([str(d) for d in dates_list])})"

    if mmsi_list is not None:
        sql += f" AND MMSI IN ({_sql_list_str([str(m) for m in mmsi_list])})"

    if segments_list is not None:
        sql += f" AND Segment IN ({_sql_list_int([int(s) for s in segments_list])})"

    if verbose:
        print("[query_ais_duckdb] SQL:\n", sql)

    con = duckdb.connect(database=":memory:")
    df = con.execute(sql).df()
    con.close()
    return df

def make_ais_folium_map(
    date: str,
    root_path: Union[str, Path] = "ais-data-parquet",
    mmsi: Optional[Union[str, Sequence[str]]] = None,
    tiles: str = "CartoDB positron",
    zoom_start: int = 8,
    verbose: bool = False,
) -> folium.Map:
    """
    Create a Folium map of AIS tracks for a given date, optionally for only a
    subset of vessels (MMSIs).

    Parameters
    ----------
    date : str
        Date to visualize (e.g. "2025-11-05").
    root_path : str or Path, optional
        Root directory of the parquet dataset (default: "ais_data_parquet").
    mmsi : str or list[str], optional
        MMSI(s) to include. If None, includes all vessels on that date.
    tiles : str, optional
        Folium tiles style (default: "CartoDB positron").
    zoom_start : int, optional
        Initial zoom level for the map.
    verbose : bool, optional
        Print some info about the data being plotted.

    Returns
    -------
    folium.Map
        A Folium map with vessel tracks drawn as polylines.

    Examples
    --------
    >>> m = make_ais_folium_map("2025-11-05")
    >>> m.save("ais_tracks_2025-11-05.html")
    >>>
    >>> m = make_ais_folium_map("2025-11-05", mmsi=["219000123", "219000456"])
    """
    # Query only what we need
    df = query_ais_duckdb(
        root_path=root_path,
        dates=date,
        mmsi=mmsi,
        columns=["MMSI", "Timestamp", "Latitude", "Longitude", "Segment"],
        verbose=verbose,
    )

    if df.empty:
        raise ValueError(f"No AIS data found for date={date} and mmsi={mmsi}")

    if verbose:
        print(
            f"[make_ais_folium_map] Loaded {len(df):,} rows, "
            f"{df['MMSI'].nunique():,} vessels"
        )

    # Ensure proper dtypes / ordering
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values(["MMSI", "Segment", "Timestamp"])

    # Map center: mean of all positions
    center_lat = df["Latitude"].mean()
    center_lon = df["Longitude"].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=tiles)

    # Group by vessel (and optionally segment)
    for (mmsi_val, seg_val), g in df.groupby(["MMSI", "Segment"]):
        coords = g[["Latitude", "Longitude"]].to_numpy().tolist()
        if len(coords) < 2:
            continue  # not enough points to draw a line

        tooltip = f"MMSI: {mmsi_val}, Segment: {seg_val}"
        folium.PolyLine(
            locations=coords,
            weight=2,
            opacity=0.8,
            tooltip=tooltip,
        ).add_to(m)

    return m


In [8]:
m = make_ais_folium_map("2025-11-05", verbose=True)
m.save("ais_tracks_2025-11-05.html")

[query_ais_duckdb] SQL:
 SELECT MMSI, Timestamp, Latitude, Longitude, Segment FROM read_parquet('ais-data-parquet\**\*.parquet') WHERE 1=1 AND Date IN ('2025-11-05')
[make_ais_folium_map] Loaded 184,183 rows, 123 vessels


In [7]:
m = make_ais_folium_map(
    date="2025-11-02",
    mmsi=["219006113", "219009229"],
    verbose=True,
)
m.save("ais_tracks_subset_2025-11-05.html")


[query_ais_duckdb] SQL:
 SELECT MMSI, Timestamp, Latitude, Longitude, Segment FROM read_parquet('ais-data-parquet\**\*.parquet') WHERE 1=1 AND Date IN ('2025-11-02') AND MMSI IN ('219006113', '219009229')
[make_ais_folium_map] Loaded 18,843 rows, 2 vessels


Loaded 1 routes.
Saved map as dkcpc_lines_bbox.html
